In [ ]:
model_name = "RF"
variation = "reduced"

n_estimators = 100
max_features = "auto"
if isinstance(max_features, int):
    print("num features per tree: ", max_features*num_features)
max_depth = 4
min_samples_leaf = 1 # default
min_samples_split = 2 #default
criterion = "gini" # 

# Let's check out a default first
clf_QT = RFC(n_jobs=-1, criterion=criterion , bootstrap=True, oob_score=True, n_estimators=n_estimators, max_features=max_features, max_depth=max_depth, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, verbose=0,)
start = time()
clf_QT.fit(X_train_GB_QT, Y_train)
traintime = time() - start
cm_rf_QT = confusion_matrix(Y_train, Y_pred)
Y_probas = cross_val_predict(clf_QT, X_train_GB_QT, Y_train, cv=5, method="predict_proba")

Y_scores = Y_probas[:, 1] # score = proba of positive class
fpr, tpr, thresholds = roc_curve(Y_train, Y_scores)

plot_roc_curve(fpr, tpr, f"Random Forest_{n_estimators}")
plt.legend(loc="lower right")
plt.show()

In [ ]:
plot_confusion_matrix(cm_rf_QT,  ["B", "G"],
                          normalize = False,
                          title = 'Confusion matrix (knn)',
                          cmap = plt.cm.Blues)
metrics_rf_QT = show_metrics(cm_rf_QT)

In [ ]:
start = time()
mets = cross_val_metrics(clf_QT, X_train_GB_QT, Y_train, explain=False)
evaltime = time() - start

#Y_pred = clf.predict(X_train)
Y_pred = cross_val_predict(clf_QT, X_train_GB_QT, Y_train, cv=5)
cm_rf_N = confusion_matrix(Y_train, Y_pred)
show_metrics(cm_rf_QT)
plot_confusion_matrix(cm_rf_QT, ["B", "G"], normalize = True, title = "Confusion matrix", cmap = plt.cm.Reds)
#plt.savefig("CM_RF_110_entropy.png")

In [ ]:
model_name = "RF"
variation = "reduced"

n_estimators = [80]
max_depth = [4]
min_samples_split = [2]

max_features = [int(0.5*num_feats), num_feats-1]

models_rf = []
params = {"n_estimators": None, "max_depth": None, "max_features": None, "min_samples_split": None}
for i,j,k in itertools.product(n_estimators, max_features, max_depth):
    print(i,j,k)
    params["n_estimators"]= i
    params["max_features"] = "auto"
    params["min_samples_split"] = j
    params["max_depth"] = k
    models_rf.append(
        (   RFC(n_jobs=-1, bootstrap=True, oob_score=True, n_estimators=i, max_features=int(j), max_depth=k, verbose=0),
            model_name+"_"+variation, 
            params.copy())
    )
print(f"# {len(models_rf)} different combinations")
cms_rf_GB_QT, res_rf_GB_QT = train_models_GB_QT(models_rf, model_name, variation) 

In [ ]:
models_rf_reduced = models_rf
res_rf_GB_QT

In [ ]:
plot_cm_per_model(cms_rf_GB_QT, models_rf_reduced)


In [ ]:
results_B = MLP_Search(alphas,activation,solver,h,maxiter,X_train_GB_QT, X_valid_GB_QT, Y_train, Y_valid)
best_params_GB_QT = FindBestScore(results_B)

In [ ]:
mlp_QT = MLPClassifier(hidden_layer_sizes=(best_params_GB_QT["h"]), max_iter=maxiter, alpha=best_params_GB_QT["alpha"],solver=best_params_GB_QT["solver"],activation=best_params_GB_QT["mode"],tol=1e-9,verbose=False)
mlp_QT.fit(X_train_GB_QT, Y_train)
Y_pred_GB_QT = mlp_QT.predict(X_valid_GB_QT)
Statistic(Y_valid,Y_pred_GB_QT,"MLP GB_QT")
cm_mlp_QT = confusion_matrix(Y_valid, Y_pred_GB_QT)

In [ ]:
plot_confusion_matrix(cm_mlp_QT, ["B", "G"],
                          normalize = False,
                          title = 'Confusion matrix (MLP)',
                          cmap = plt.cm.Blues)
metrics_mlp_QT = show_metrics(cm_mlp_QT)

In [ ]:
model_name = "MLP"
variation = "reduced"

models_mlp = []
params = {"h": None, "alphas": None, "activations": None}
for i,j,k in itertools.product(h, alphas, activation):
    print(i,j,k)
    params["alphas"]= i
    params["activation"] = "auto"
    params["min_samples_split"] = j
    params["max_depth"] = k
    models_mlp.append(
        (MLPClassifier(hidden_layer_sizes=(i), max_iter=maxiter, alpha=j,solver="adam",activation=k,tol = 1e-9),
         model_name+"_"+variation, 
         params.copy())
    )
print(f"# {len(models_mlp)} different combinations")

cms_mlp_GB_QT, res_mlp_GB_QT = train_models_GB_QT(models_mlp, model_name, variation, x=X_train_GB_QT, y=Y_train)   
models_mlp_reduced = models_mlp

In [ ]:
res_mlp_GB_QT

In [ ]:
# KNN Test
from sklearn.neighbors import KNeighborsClassifier

scores = []
best_score = 0
best_ks = []
ks = list(range(1, 10)) + list(range(10, 20, 2)) + list(range(20, 51, 4))
for k in ks:
    knn_QT = KNeighborsClassifier(n_neighbors=k)
    knn_QT.fit(X_train_GB_QT, Y_train)
    Y_pred = knn_QT.predict(X_train_GB_QT)
    score = accuracy_score(Y_train, Y_pred)
    scores.append(score)
    best_score = max(scores)
    if score == best_score:
        best_ks.append(k)
    score
best_score
best_ks.pop(0)
#best_ks.pop(0)
#best_ks

In [ ]:
plt.plot(ks, scores, label="scores")
plt.plot(best_ks, [best_score]*len(best_ks), marker="o", color="red",label="best")
plt.legend()



knn_QT = KNeighborsClassifier(n_neighbors=5)
knn_QT.fit(X_train_GB_QT, Y_train)
Y_pred = knn_QT.predict(X_train_GB_QT)
score_knn_QT = accuracy_score(Y_train, Y_pred)
cm_knn_QT = confusion_matrix(Y_train, Y_pred)
cm_knn_QT

In [ ]:
# On train set
print("train set")
cross_val_metrics(knn_QT, X_train_GB_QT, Y_train)
# On traination set
print("-"*30)
print("traination set")
cross_val_metrics(knn_QT, X_train_GB_QT, Y_train)
# B...1 --> 66%
# M...0 --> 33%

In [ ]:
plot_confusion_matrix(cm_knn_QT,  ["B", "G"],
                          normalize = False,
                          title = 'Confusion matrix (knn)',
                          cmap = plt.cm.Blues)
metrics_knn_QT = show_metrics(cm_knn_QT)

In [ ]:
# KNN Test
from sklearn.neighbors import KNeighborsClassifier

ks = list(range(1, 10)) + list(range(10, 20, 2)) + list(range(20, 51, 4))
print(ks)
weights = ["uniform", "distance"]
metrics = []
for weight in weights:
    scores = []
    mets = []
    best_ks = []
    best_k = 0
    best_score = 0
    for k in ks:
        knn = KNeighborsClassifier(n_neighbors=k, weights=weight)
        #knn.fit(X_train, Y_train)
        #Y_pred = knn.predict(X_valid)
        #score = accuracy_score(Y_valid, Y_pred)
        metric = cross_val_metrics(knn, X_train_GB_QT, Y_train, cv=5, show=False)
        mets.append(metric)
        score = metric["accuracy"]
        scores.append(score)
        #best_score = max(scores)
        if score > best_score:
            best_k = k
            best_score= score
        score
    metrics.append(mets)
    best_score
    best_ks
    plt.clf()
    plt.figure(figsize=(8,6))
    plt.suptitle(f"Accuracy score over k (weights={weight})")
    plt.plot(ks, scores, label="accuracy")
    plt.plot(ks, [met["f1"] for met in mets], label="f1")
    plt.plot(best_k, best_score, marker="o", color="red", label="best")
    plt.legend()
    plt.show()
    plt.savefig(f"figures//wine//knn//scores_k_{weight}_reduced")